In [1]:
import eleanor
import lightkurve as lk 
from IPython.display import clear_output
import pandas as pd
import numpy as np
import pickle
from os import listdir
import warnings

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [20]:
warnings.filterwarnings('ignore')
file_list = []
for files in listdir('F:/lightcurves'):
    file_list.append(int(files[:-9]))
total_targets = range(0,len(pd.read_csv('F:/catalogues/final_quasar_catalogue.csv', sep=',', header=0, usecols=['RA'],squeeze=True)))
target_list = [x for x in total_targets if x not in file_list]
print(target_list[:30])
print(len(target_list)/len(total_targets))

[195, 627, 981, 1472, 2910, 3613, 3643, 3670, 3716, 3763, 3802, 3855, 3875, 3949, 4146, 4157, 4215, 4241, 4482, 4518, 4609, 4648, 4768, 4887, 5332, 5362, 5391, 5431, 5556, 5561]
0.33785957607872824


In [ ]:
ral = pd.read_csv('F:/catalogues/final_quasar_catalogue.csv', sep=',', header=0, usecols=['RA'],squeeze=True)
decl = pd.read_csv('F:/catalogues/final_quasar_catalogue.csv', sep=',', header=0, usecols=['DEC'],squeeze=True)
terrible_targets = []

for row in target_list[10:]:
    try:
        ra = ral[row]
        dec = decl[row]

        star = eleanor.multi_sectors(sectors='all',tic=0,coords=(ra,dec),tc=True)

        full_lc = lk.LightCurve([],[])
        for observation in star:
            data = eleanor.TargetData(observation,bkg_size=15,do_pca=False)
            data.custom_aperture(shape='rectangle', h=1,w=1, method='exact')
            data.get_lightcurve()

            q = data.quality == 0

            lc = data.to_lightkurve()#flux=data.pca_flux)
            lc.flux = lc.flux - np.median(lc.flux)
            full_lc = full_lc.append(lc)

            with open('F:/sector_lightcurves/' + str(row) + '-' + str(observation.sector) + 'lc.pickle','wb') as wfile:
                pickle.dump(lc,wfile)
        with open('F:/lightcurves/' + str(row) + 'lc.pickle','wb') as wfile:
            pickle.dump(full_lc,wfile)
        clear_output()
        print("Done")
    except:
        terrible_targets.append(row)
        clear_output()
        print("Error")
        continue

Done
Found star in Sector(s) 14 15 21
Inflating...
Inflating...

In [12]:
from tess_stars2px import tess_stars2px_function_entry as tess_stars2px
result = tess_stars2px(8675309, ra, dec)
sector = result[3][result[3] < 21.5]
sectors = sector.tolist()
len(sectors)

0